In [60]:
import cv2
import numpy as np
from scipy.signal import convolve2d
from scipy.ndimage.filters import correlate 
import matplotlib.pyplot as plt
import matplotlib.image as pltimg 
from scipy.spatial.distance import cdist
from scipy.spatial import distance
from scipy import *
from scipy import linalg
from scipy import ndimage



In [61]:
def load_gray_to_double(path):
    '''
    :param path: path to load image
    :return: grayscaled and float image
    '''
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    # transmit the image to float
    img = img.astype(np.float64)/255.0

    print(img.shape)
    return img

def load_rgb(path):
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    return img


def show_img(img):
    plt.imshow(img, cmap='gray')
    plt.show()
    
    
def normalize(img):
    ''' Function to normalize an input array to 0-1 '''
    img_min = img.min()
    img_max = img.max()
    return (img - img_min) / (img_max - img_min)

In [63]:
"""
Harris Corner Detector
Usage: Call the function harris(filename) for corner detection
Reference   (Code adapted from):
             http://www.kaij.org/blog/?p=89
             Kai Jiang - Harris Corner Detector in Python
             
"""
from pylab import *
from scipy import signal
from scipy import *
import numpy as np
from PIL import Image

def harris(filename, min_distance = 10, threshold = 0.1):
    """
    filename: Path of image file
    threshold: (optional)Threshold for corner detection
    min_distance : (optional)Minimum number of pixels separating 
     corners and image boundary
    """
    im = np.array(Image.open(filename).convert("L"))
    harrisim = compute_harris_response(im)
    filtered_coords = get_harris_points(harrisim, min_distance, threshold)
    plot_harris_points(im, filtered_coords)
    return filtered_coords

def gauss_derivative_kernels(size, sizey=None):
    """ returns x and y derivatives of a 2D 
        gauss kernel array for convolutions """
    size = int(size)
    if not sizey:
        sizey = size
    else:
        sizey = int(sizey)
    y, x = mgrid[-size:size+1, -sizey:sizey+1]
    #x and y derivatives of a 2D gaussian with standard dev half of size
    # (ignore scale factor)
    gx = - x * exp(-(x**2/float((0.5*size)**2)+y**2/float((0.5*sizey)**2))) 
    gy = - y * exp(-(x**2/float((0.5*size)**2)+y**2/float((0.5*sizey)**2))) 
    return gx,gy

def gauss_kernel(size, sizey = None):
    """ Returns a normalized 2D gauss kernel array for convolutions """
    size = int(size)
    if not sizey:
        sizey = size
    else:
        sizey = int(sizey)
    x, y = mgrid[-size:size+1, -sizey:sizey+1]
    g = exp(-(x**2/float(size)+y**2/float(sizey)))
    return g / g.sum()

def compute_harris_response(im):
    """ compute the Harris corner detector response function 
        for each pixel in the image"""
    #derivatives
    gx,gy = gauss_derivative_kernels(3)
    imx = signal.convolve(im,gx, mode='same')
    imy = signal.convolve(im,gy, mode='same')
    #kernel for blurring
    gauss = gauss_kernel(3)
    #compute components of the structure tensor
    Wxx = signal.convolve(imx*imx,gauss, mode='same')
    Wxy = signal.convolve(imx*imy,gauss, mode='same')
    Wyy = signal.convolve(imy*imy,gauss, mode='same')   
    #determinant and trace
    Wdet = Wxx*Wyy - Wxy**2
    Wtr = Wxx + Wyy   
    return Wdet / Wtr

def get_harris_points(harrisim, min_distance=10, threshold=0.1):
    """ return corners from a Harris response image
        min_distance is the minimum nbr of pixels separating 
        corners and image boundary"""
    #find top corner candidates above a threshold
    corner_threshold = max(harrisim.ravel()) * threshold
    harrisim_t = (harrisim > corner_threshold) * 1    
    #get coordinates of candidates
    candidates = harrisim_t.nonzero()
    coords = [ (candidates[0][c],candidates[1][c]) for c in range(len(candidates[0]))]
    #...and their values
    candidate_values = [harrisim[c[0]][c[1]] for c in coords]    
    #sort candidates
    index = argsort(candidate_values)   
    #store allowed point locations in array
    allowed_locations = zeros(harrisim.shape)
    allowed_locations[min_distance:-min_distance,min_distance:-min_distance] = 1   
    #select the best points taking min_distance into account
    filtered_coords = []
    for i in index:
        if allowed_locations[coords[i][0]][coords[i][1]] == 1:
            filtered_coords.append(coords[i])
            allowed_locations[(coords[i][0]-min_distance):(coords[i][0]+min_distance),
                (coords[i][1]-min_distance):(coords[i][1]+min_distance)] = 0               
    return filtered_coords

def plot_harris_points(image, filtered_coords):
    """ plots corners found in image"""
    figure()
    gray()
    imshow(image)
    plot([p[1] for p in filtered_coords],[p[0] for p in filtered_coords],'r*')
    axis('off')
    show()

#harris('./CS543_ECE549 Assignment 3_files/sample_panorama.JPG')

In [64]:
def warp_images(image0, image1, transform):
    r, c = image1.shape[:2]
    # Note that transformations take coordinates in (x, y) format,
    # not (row, column), in order to be consistent with most literature
    corners = np.array([[0, 0],
                        [0, r],
                        [c, 0],
                        [c, r]])

    # Warp the image corners to their new positions
    warped_corners = transform(corners)

    # Find the extents of both the reference image and the warped
    # target image
    all_corners = np.vstack((warped_corners, corners))

    corner_min = np.min(all_corners, axis=0)
    corner_max = np.max(all_corners, axis=0)

    output_shape = (corner_max - corner_min)
    output_shape = np.ceil(output_shape[::-1])

    offset = SimilarityTransform(translation=-corner_min)

    image0_ = warp(image0, offset.inverse, output_shape=output_shape, cval=-1)

    image1_ = warp(image1, (transform + offset).inverse, output_shape=output_shape, cval=-1)

    image0_zeros = warp(image0, offset.inverse, output_shape=output_shape, cval=0)

    image1_zeros = warp(image1, (transform + offset).inverse, output_shape=output_shape, cval=0)

    overlap = (image0_ != -1.0 ).astype(int) + (image1_ != -1.0).astype(int)
    overlap += (overlap < 1).astype(int)
    merged = (image0_zeros+image1_zeros)/overlap

    im = Image.fromarray((255*merged).astype('uint8'), mode='RGB')
    im.save('stitched_images.jpg')
    im.show()

In [65]:
"""
    Extract  descriptor
    returns a descriptor of size numoffeature * (neighbor+1)*(neighbor+1)
"""
def descriptor_extract(img, neighborhoods, feature_points):
    num_Of_points = len(feature_points)
    print(num_Of_points)
    descriptors = np.zeros((num_Of_points, (2*neighborhoods+1)**2))
    tmp_img = img.copy()
    
    for i in range(num_Of_points):
        # obtain img slices
        # !! Remember to +1 for slice right part! 
        tmp_img = img[feature_points[i][0]-neighborhoods:
                      feature_points[i][0]+neighborhoods+1,
                  feature_points[i][1]-neighborhoods:
                  feature_points[i][1]+neighborhoods+1]
        descriptors[i, :] = tmp_img.reshape((1, (2*neighborhoods+1)**2))
        
    return descriptors

In [68]:
Matrix = []
sample_numbers = 4
matches = np.random.randint(1,300,(300,4))
inliers = np.random.choice(300,4)
for j in range(sample_numbers):
    current_match = matches[inliers[j]]
    #left feature point
    xt = np.array([current_match[1], current_match[0], 1]).T
    new_row = np.append(np.append(xt*0, xt), xt*-current_match[2])
    # new_row = np.array([0*xt,xt,-current_match[2]*xt])
    new_row2 =np.append(np.append(xt, xt * 0), xt * -current_match[3])
    if len(Matrix) == 0:
        Matrix = new_row
    else:
        Matrix = np.vstack([Matrix, new_row])
    Matrix = np.vstack([Matrix, new_row2])

print(Matrix)
_,_,VMatrix = np.linalg.svd(Matrix)
HMatrix = VMatrix[len(VMatrix)-1]
H_reshape = np.reshape(HMatrix, (3, 3))
print(H_reshape)

def dist2(x,y):   
    return np.sqrt(np.sum((x-y)**2))

# RANSAC
# matches: numberof features * 2 coords
# ((points_left_x,points_right_y,points_right_x,points_right_y))
def RANSAC(matches, threshold = 10, iterations = 250, good_model_num = 15):
    sample_numbers = 4
    match_number = len(matches)
    print("num of matches", match_number)
    # start number for each iteration
    cnt = 1
    model_error = 255
    while cnt < iterations: 
        # get 4 random samples from matches
        if sample_numbers == 4:
            # get sample from matches
            inliers = np.random.choice(match_number, sample_numbers)
        

       
        A = np.array([])
        # for each sample:
        for j in range(sample_numbers):
            current_match = matches[inliers[j]]
            #left feature point
            xt = np.array([current_match[1], current_match[0], 1]).T
            new_row = np.append(np.append(xt*0, xt), xt*-current_match[2])
            # new_row = np.array([0*xt,xt,-current_match[2]*xt])
            new_row2 =np.append(np.append(xt, xt * 0), xt * -current_match[3])
            if len(A) == 0:
                A = new_row
            else:
                A = np.vstack([A, new_row])
            A = np.vstack([A, new_row2])
        
        # Homography fitting
        _,_,VMatrix = np.linalg.svd(A)
        HMatrix = VMatrix[len(VMatrix)-1]
        H_reshape = np.reshape(HMatrix, (3, 3))
        
        
        
        # consensus_set = []
        # for i in range(len(matches)):
        #     x1,y1 = matches[i][0],matches[i][1]
        #     x2,y2 = matches[i][2],matches[i][3]
        #     A = array([x1, y1, 1]).reshape(3,1)
        #     B = array([x2, y2, 1]).reshape(3,1)
        #     out = B - dot(H_reshape, A)
        #     dist_err = hypot(out[0][0], out[1][0])
        #     if dist_err < threshold:
        #         consensus_set.append(matches[i])
        #         
        # # Check how well is our speculated model
        # if len(consensus_set) >= good_model_num:
        #     dists = []
        #     for p in consensus_set:
        #         x0, y0 = p[0],p[1]
        #         x1, y1 = p[2],p[3]
        # 
        #         A = array([x0, y0, 1]).reshape(3,1)
        #         B = array([x1, y1, 1]).reshape(3,1)
        # 
        #         out = B - dot(H_reshape, A)
        #         dist_err = hypot(out[0][0], out[1][0])
        #         dists.append(dist_err)
        #     if (max(dists) < threshold) and (max(dists) < model_error):
        #         model_error = max(dists)
        #         model_H = H_reshape
        # if model_H is None:
        #     print("No result!")
        # else:
        #     return model_H
        
        # print(H_reshape)
        inliers_number = 0
        inliers = []
        residual = []
        for i in range(len(matches)):
            right_tmp =  np.array([matches[i][1],matches[i][0], 1])
            X = np.matmul(np.transpose(H_reshape),
                            right_tmp)
            x, y= X[0]/X[2],X[1]/X[2]

            sample_point = np.array([x, y])
            match_point = np.array([matches[i][3], matches[i][2]])
            # print(sample_point)
            # print(match_point)
            tmpDis = np.linalg.norm(sample_point-match_point)

            # print(tmpDis)
            if tmpDis < threshold:
                inliers += [i]
                residual += [tmpDis]
                inliers_number += 1

        if inliers_number < good_model_num:
            sample_numbers = 4
            # print("inliernumber:",inliers_number)
        else:
            sample_numbers = inliers_number
            cnt += 1
            # print("got one")

    print("num of iterations:",cnt)
    mean_of_residual = np.mean(residual)
    print(inliers_number)    
    if H_reshape is None:
        print("no model got")
    return  H_reshape



[[     0      0      0    109     37      1 -30193 -10249   -277]
 [   109     37      1      0      0      0 -23653  -8029   -217]
 [     0      0      0    135     48      1 -22275  -7920   -165]
 [   135     48      1      0      0      0 -31455 -11184   -233]
 [     0      0      0    134    289      1  -2948  -6358    -22]
 [   134    289      1      0      0      0 -13266 -28611    -99]
 [     0      0      0    235     71      1 -66740 -20164   -284]
 [   235     71      1      0      0      0 -25850  -7810   -110]]


[[ 4.54770610e-03 -5.58058075e-03 -4.80399703e-01]
 [ 6.12870951e-03 -9.48045825e-04 -8.76994036e-01]
 [ 3.69461232e-05 -5.96446869e-05 -2.70129349e-03]]


In [72]:
def show_features(img_l,img_r,matches,final_matches):
    implot_l = plt.imshow(img_l)
    print(len(matches))
    print(len(final_matches))
    for i in range(len(matches)):
        plt.scatter(x=matches[i][1], y=matches[i][0], c='r', s=10)
    for i in range(len(final_matches)):
        plt.scatter(x=final_matches[i][1], y=final_matches[i][0], c='g', s=10)

    plt.show()
    
    implot_r = plt.imshow(img_r)
    for i in range(len(matches)):
        plt.scatter(x=matches[i][3], y=matches[i][2], c='r', s=10)
    for i in range(len(final_matches)):
        plt.scatter(x=final_matches[i][3], y=final_matches[i][2], c='g', s=10)

    plt.show()


In [73]:
def gauss_img(img, sigma):
    print(img.shape)
    tmp_img = img
    kernel = gauss_kernel(2*sigma+1)
    print(img.ndim)
    if img.ndim == 3:
        for i in range(img.ndim):
            tmp_img[:, :, i] = correlate(tmp_img[:, :, i], kernel)
    else:
        tmp_img = correlate(tmp_img, kernel)
    return tmp_img
        

In [76]:


def Haffine_from_points(fp,tp):
    """ 
        Find affine transform
    """

    if fp.shape != tp.shape:
        raise RuntimeError

    #condition points
    #-from points-
    m = mean(fp[:2], axis=1)
    maxstd = max(std(fp[:2], axis=1))
    C1 = diag([1/maxstd, 1/maxstd, 1])
    C1[0][2] = -m[0]/maxstd
    C1[1][2] = -m[1]/maxstd
    fp_cond = dot(C1,fp)

    #-to points-
    m = mean(tp[:2], axis=1)
    C2 = C1.copy() #must use same scaling for both point sets
    C2[0][2] = -m[0]/maxstd
    C2[1][2] = -m[1]/maxstd
    tp_cond = dot(C2,tp)

    #conditioned points have mean zero, so translation is zero
    A = concatenate((fp_cond[:2],tp_cond[:2]), axis=0)
    U,S,V = linalg.svd(A.T)

    #create B and C matrices as Hartley-Zisserman (2:nd ed) p 130.
    tmp = V[:2].T
    B = tmp[:2]
    C = tmp[2:4]

    tmp2 = concatenate((dot(C,linalg.pinv(B)),zeros((2,1))), axis=1)
    H = vstack((tmp2,[0,0,1]))

    #decondition
    H = dot(linalg.inv(C2),dot(H,C1))

    return H / H[2][2]

def get_homography(points_list):
    '''
        Function to quickly compute a homography matrix from all point 
        correspondences.
        Inputs:
            points_list: tuple of tuple of tuple of correspondence indices. Each
            entry is [[x1, y1], [x2, y2]] where [x1, y1] from image 1 corresponds
            to [x2, y2] from image 2.
        Outputs:
            H: Homography matrix.
    '''
    fp = ones((len(plist), 3))
    tp = ones((len(plist), 3))

    for idx in range(len(plist)):
        fp[idx, 0] = plist[idx][0][0]
        fp[idx, 1] = plist[idx][0][1]

        tp[idx, 0] = plist[idx][1][0]
        tp[idx, 1] = plist[idx][1][1]

    H = Haffine_from_points(fp.T, tp.T)

    return H

from random import choice


def ransac(im1, im2, points_list, iters = 500 , error = 10, good_model_num = 5):
    if ndim(im1) == 2:
        rows,cols = im1.shape
    else:
        rows, cols, _ = im1.shape
    model_error = 255
    model_H = None

    for i in range(iters):
        consensus_set = []
        points_list_temp = copy(points_list).tolist()
        # Randomly select 3 points
        for j in range(3):
            temp = choice(points_list_temp)
            consensus_set.append(temp)
            points_list_temp.remove(temp)
            
        # Calculate the homography matrix
    
        fp0 = []
        fp1 = []
        fp2 = []
    
        tp0 = []
        tp1 = []
        tp2 = []
        for line in consensus_set:
    
            fp0.append(line[0][0])
            fp1.append(line[0][1])
            fp2.append(1)
    
            tp0.append(line[1][0])
            tp1.append(line[1][1])
            tp2.append(1)
    
        fp = array([fp0, fp1, fp2])
        tp = array([tp0, tp1, tp2])
    
        H = Haffine_from_points(fp, tp)
    
        # Transform the second image
        # imtemp = transform_im(im2, [-xshift, -yshift], -theta)
        # Check if the other points fit this model
    
        for p in points_list_temp:
            x1, y1 = p[0]
            x2, y2 = p[1]
    
            A = array([x1, y1, 1]).reshape(3,1)
            B = array([x2, y2, 1]).reshape(3,1)
    
            out = B - dot(H, A)
            dist_err = hypot(out[0][0], out[1][0])
            if dist_err < error:
                consensus_set.append(p)
    
    
        # Check how well is our speculated model
        if len(consensus_set) >= good_model_num:
            dists = []
            for p in consensus_set:
                x0, y0 = p[0]
                x1, y1 = p[1]
    
                A = array([x0, y0, 1]).reshape(3,1)
                B = array([x1, y1, 1]).reshape(3,1)
    
                out = B - dot(H, A)
                dist_err = hypot(out[0][0], out[1][0])
                dists.append(dist_err)
            if (max(dists) < error) and (max(dists) < model_error):
                model_error = max(dists)
                model_H = H

    return model_H

def affine_transform2(im, rot, shift):
    '''
        Perform affine transform for 2/3D images.
    '''
    if ndim(im) == 2:
        return ndimage.affine_transform(im, rot, shift)
    else:
        imr = ndimage.affine_transform(im[:, :, 0], rot, shift)
        img = ndimage.affine_transform(im[:, :, 1], rot, shift)
        imb = ndimage.affine_transform(im[:, :, 2], rot, shift)

        return dstack((imr, img, imb))
    
smoothing_window_size = 800
def create_mask(img1,img2,version):
        height_img1 = img1.shape[0]
        width_img1 = img1.shape[1]
        width_img2 = img2.shape[1]
        height_panorama = height_img1
        width_panorama = width_img1 +width_img2
        offset = int(smoothing_window_size / 2)
        barrier = img1.shape[1] - int(smoothing_window_size / 2)
        mask = np.zeros((height_panorama, width_panorama))
        if version== 'left_image':
            mask[:, barrier - offset:barrier + offset ] = np.tile(np.linspace(1, 0, 2 * offset ).T, (height_panorama, 1))
            mask[:, :barrier - offset] = 1
        else:
            mask[:, barrier - offset :barrier + offset ] = np.tile(np.linspace(0, 1, 2 * offset ).T, (height_panorama, 1))
            mask[:, barrier + offset:] = 1
        return cv2.merge([mask, mask, mask])

In [78]:
img_1_left_path = "data\\part1\\left.jpg"
img_1_right_path = "data\\part1\\right.jpg"

In [80]:
# load image
img1_left_rgb = load_rgb(img_1_left_path)
img1_right_rgb = load_rgb(img_1_right_path)
img1_left_gray = load_gray_to_double(img_1_left_path)
img1_right_gray = load_gray_to_double(img_1_right_path)
# img1_left = load_image(img_1_left_path)
# img1_right = load_image(img_2_right_path)
# show_img(img1_left)
# show_img(img1_right)
neighborhoods = 5

# harris Cornor detect feature points 
feature_points_left = harris(img_1_left_path, min_distance=2*neighborhoods)
feature_points_right = harris(img_1_right_path, min_distance=2*neighborhoods)
# print(feature_points_left)




(398, 800)
(398, 800)


C:\Users\bluerain\PycharmProjects\cvmp2\new_venv\lib\site-packages\ipykernel_launcher.py:70: RuntimeWarning: divide by zero encountered in true_divide


In [83]:
"""
Extract local neighborhoods around every keypoint in both images
and form descriptors simply by "flattening" the pixel values in each neighborhood 
to one-dimensional vectors.
"""
from scipy.stats import zscore

descriptor_left = descriptor_extract(img1_left_gray, 
                                      neighborhoods, feature_points_left)
descriptor_right = descriptor_extract(img1_right_gray, 
                                       neighborhoods, feature_points_right)
#Normalize to zero mean
norm_descriptor_left = zscore(descriptor_left)

norm_descriptor_right = zscore(descriptor_right)


598
376


In [84]:
from scipy.spatial.distance import cdist
"""
Compute distances between every descriptor in one image and every descriptor in the other image. 
In Python, you can use scipy.spatial.distance.cdist(X,Y,'sqeuclidean') 
for fast computation of Euclidean distance. 
"""
# dim of distance: num_Of_left * num_of_right
distance = cdist(norm_descriptor_left, norm_descriptor_right, 'sqeuclidean')
print(distance.shape)
"""
Select putative matches based on the matrix of pairwise descriptor distances obtained above. 
You can select all pairs whose descriptor distances are below a specified threshold, 
or select the top few hundred descriptor pairs with the smallest pairwise distances.
"""

# num of points to extract 
total_select_numbers = min(300, len(feature_points_right), len(feature_points_left))
select_match = []
INF = 1111111
for i in range(total_select_numbers):
    # find matrix minimum index
    ri, ci = np.unravel_index(distance.argmin(), distance.shape)
    select_match += [(feature_points_left[ri][0],feature_points_left[ri][1],
                      feature_points_right[ci][0],feature_points_right[ci][1])]
    # set to INF after used
    distance[ri, :] = INF
    distance[:, ci] = INF

    
"""
Implement RANSAC to estimate a homography mapping one image onto the other. 
Report the number of inliers and the average residual for the inliers 
(squared distance between the point coordinates in one image and
 the transformed coordinates of the matching point in the other image). 
Also, display the locations of inlier matches in both images.
"""
# [inliers, inliers_number, mean_of_residual, H] = RANSAC(select_match,
#                                                         threshold=220,
#                                                         good_model_num=10)
# final_Match = []
# print("final match number is",inliers_number)
# for i in range(inliers_number):
#     final_Match += [select_match[inliers[i]]]
# 
# # show features
# show_features(img1_left_rgb,img1_right_rgb,
#               select_match,  final_Match)

# points_list = [[[matches[0],matches[1]],[matches[2],matches[3]]] for matches in select_match]
# out_ransac = ransac(img1_left_rgb,img1_right_gray,points_list,good_model_num=6)

out_ransac = RANSAC(select_match,iterations=300, threshold=255,good_model_num=10)
print(out_ransac)
# H_ransac = inv(out_ransac)
H_ransac = out_ransac
warp_images(img1_left_rgb,img1_left_rgb,H_ransac)
height_img1 = img1_left_rgb.shape[0]
width_img1 = img1_left_gray.shape[1]
width_img2 = img1_right_gray.shape[1]
height_panorama = height_img1
width_panorama = width_img1 +width_img2

panorama1 = np.zeros((height_panorama, width_panorama, 3))
mask1 = create_mask(img1_left_rgb, img1_right_rgb,version='left_image')
panorama1[0:img1_left_rgb.shape[0], 0:img1_left_rgb.shape[1], :] = img1_left_rgb
panorama1 *= mask1
mask2 = create_mask(img1_left_rgb, img1_right_rgb,version='right_image')
panorama2 = cv2.warpPerspective(img1_right_rgb, H_ransac, (width_panorama, height_panorama))*mask2
result= panorama1+panorama2

rows, cols = np.where(result[:, :, 0] != 0)
min_row, max_row = min(rows), max(rows) + 1
min_col, max_col = min(cols), max(cols) + 1
final_result = result[min_row:max_row, min_col:max_col, :]
show_img(normalize(final_result))
print(final_result.shape)

(598, 376)
num of matches 300
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one


got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one


got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one


got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one


got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one

...

got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one


got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one


got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one


got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
got one
num of iterations: 300
23
[[-3.51860781e-04 -1.44205803e-03  4.55366065e-01]
 [-7.65155172e-04 -2.99741993e-03  8.90285462e-01]
 [-4.11181898e-06 -1.51708916e-05  4.66583266e-03]]


TypeError: 'numpy.ndarray' object is not callable